## Evaluation of Models on Incident Prediction

In [1]:
import numpy as np
import torch
import pandas as pd
from torchmetrics import Accuracy
# from torchmetrics.functional import precision_recall
from tqdm import tqdm
from datetime import datetime as dt
from datetime import time
from matplotlib import pyplot as plt
from copy import deepcopy
from torch.utils.data import DataLoader, Dataset, Subset
from GAN_GBM import create_parser
import matplotlib.dates as mdates
from datetime import timedelta
from scipy.stats import gumbel_r
from matplotlib.dates import DateFormatter

c:\Users\haowu\Anaconda3\envs\traffic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\haowu\Anaconda3\envs\traffic\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
((3000000000/6368)*108.12)/(3600*24)

589.5362227805696

In [ ]:
from html2text import HTML2Text

def html_to_text(html: str) -> str:
    """Converts HTML content to plain text.
    Args:
        html (bytes): HTML content as bytes.
    Returns:
        str: Plain text extracted from HTML.
    """
    html = html.decode('latin1')
    h = HTML2Text()
    h.ignore_links = False
    text = h.handle(html)
    return text

# Example HTML content to run through the function
html_example = b"""
<html>
<head>
    <title>Sample HTML</title>
</head>
<body>
    <h1>Main Title</h1>
    <h2>Subtitle</h2>
    <p>This is a paragraph with <a href="https://example.com">a link</a> and an image below.</p>
    <img src="https://example.com/image.png" alt="Example Image">
    <table>
        <tr><th>Header 1</th><th>Header 2</th></tr>
        <tr><td>Cell 1</td><td>Cell 2</td></tr>
    </table>
</body>
</html>
"""

# Run the function on the example HTML content
plain_text = html_to_text(html_example)
print(plain_text)

: 

# 0. Load Args

In [2]:
parser = create_parser()
args = parser.parse_args(args=[])
# For reproducibility

In [3]:
args 

Namespace(model_type=None, dropout_prob=0.3, seq_len_in=7, seq_len_out=6, freq_out=5, inc_threshold=0.5, LR_pos_weight=100, nonrec_spd_weight=15, use_expectation=False, use_gt_inc=0, training_granularity=1, time_series_cv_ratio=1, data_train_ratio=0.7, data_val_ratio=0.2, seed=3407, dim_in=0, dim_out=1, num_node=207, use_spd_all=True, use_spd_truck=False, use_spd_pv=False, use_slowdown_spd=True, use_tti=True, use_HA=False, use_dens=False, use_weather=False, use_time=False, use_waze=False, county='TSMO', link_id='110+04483', number_of_point_per_day=186, number_of_business_day=260, upstream_range_mile=2, downstream_range_mile=1, inc_ahead_label_min=0, task=None, num_epochs=200, batch_size=64, num_workers=0, lr=2.5e-05, nz=100, beta1=0.9, beta2=0.99, exp_name=None, data_dir='E:/two_stage_model', log_dir='./logs', checkpoint_dir='./checkpoints', checkpoint_every=100, load_checkpoint='', load_checkpoint_epoch=-1)

In [4]:
torch.cuda.is_available() # Check if GPU is used

True

# 1. Load Data

In [5]:
def load_SVM_pred(use_GAN, args):
    if use_GAN:
        pred = np.load(f'{args.data_dir}/model/GAN_baseline/{args.county}/{args.link_id}/svm_gan_inc.npy')
    else:
        pred = np.load(f'{args.data_dir}/model/GAN_baseline/{args.county}/{args.link_id}/svm_inc.npy')
    if args.county == 'TSMO':
        pred = pred.reshape(26, 174)
    else:
        pred = pred.reshape(51, 174)
    return pred


In [6]:
def load_test_annomaly(args):
    df_select_inc = pd.read_pickle(f"{args.data_dir}/data/{args.county}/processed_data/{args.county}_selected_incident.pkl")
    link_annomaly = np.array(df_select_inc[args.link_id])
    if args.county == 'TSMO':
        link_annomaly = link_annomaly.reshape(260, 186)
        test_annomaly = link_annomaly[-26:, 7:-5]
    else:
        link_annomaly = link_annomaly.reshape(522, 186)
        test_annomaly = link_annomaly[-51:, 7:-5]
    return test_annomaly

# 2. Load Spd

In [72]:
def plot_speed_for_time_range(args, plot_name, link_id, start_time, end_time, ahead_number, next_number, shadow, red_hatch, compare_days):
    speed_df = pd.read_pickle(f"{args.data_dir}/data/{args.county}/processed_data/{args.county}_df_spd_tmc_5min_all_from_1_min.pkl")

    
    speed_df.reset_index(inplace=True)
    speed_df.rename(columns={'index': 'time'}, inplace=True)
    speed_df['time'] = pd.to_datetime(speed_df['measurement_tstamp'])
    link_df = speed_df[[link_id, 'time']]

    # 找到给定时间段的索引
    closest_start_idx = (link_df['time'] - start_time).abs().argsort().iloc[0]
    closest_end_idx = (link_df['time'] - end_time).abs().argsort().iloc[0]

    # 向上取两个最近的五分钟
    start_idx = max(closest_start_idx - ahead_number, 0)
    end_idx = min(closest_end_idx + next_number, len(link_df) - 1)
    
    # 取出对应的时间段
    plot_df = link_df.iloc[start_idx:end_idx+1]

    plt.figure(figsize=(12, 6))
    plt.plot(plot_df['time'], plot_df[link_id], marker='o', label=f'{start_time.date()} Speed')
    plt.title(f"{plot_name}")
    plt.xlabel('Time')
    plt.ylabel('Speed')
    plt.grid(True)
    
    if shadow:
        plt.axvspan(shadow[0], shadow[1], color='red', alpha=0.3,  label=f'{start_time.date()} Incident Report')

    if red_hatch:
        plt.fill_betweenx(y=[plot_df[link_id].min(), plot_df[link_id].max()], x1=red_hatch[0], x2=red_hatch[1],
                          edgecolor='red', facecolor='none', alpha=0.3, hatch='//', label=f'{start_time.date()} Alert')

    ax = plt.gca()
    ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))  # 仅显示时刻和分钟

    # 添加比较的折线图
    for days in compare_days:
        compare_start_time = start_time + timedelta(days=days)
        compare_end_time = end_time + timedelta(days=days)
        print(compare_start_time, compare_end_time)
        print(compare_start_time.day)
        
        compare_closest_start_idx = (link_df['time'] - compare_start_time).abs().argsort().iloc[0]
        compare_closest_end_idx = (link_df['time'] - compare_end_time).abs().argsort().iloc[0]

        compare_start_idx = max(compare_closest_start_idx - ahead_number, 0)
        compare_end_idx = min(compare_closest_end_idx + next_number, len(link_df) - 1)
        
        compare_plot_df = link_df.iloc[compare_start_idx:compare_end_idx+1]
        plt.plot(plot_df['time'], compare_plot_df[link_id],  marker='x', linestyle='--', label=f'{compare_start_time.date()} Speed')
        
        # 规范化时间
        #compare_plot_df['norm_time'] = (compare_plot_df['time'] - compare_plot_df['time'].iloc[0]).dt.total_seconds() / 60

        #plt.plot(compare_plot_df['norm_time'], compare_plot_df[link_id], marker='x', linestyle='--', label=f'Comparison {days} days')
    
    plt.ylim(10, 75)

    plt.legend()
    plt.show()

# 3. Incident Evaluation Function

In [148]:
class IncidentDetectionAnalysis:
    
    def __init__(self, 
            args,
            prediciotn_array, 
            annomaly_array,
            interval =  timedelta(minutes=5), 
            prediction_start_time =  '06:00', #'06:05',
        ):
        self.link_id = args.link_id # tmc id link
        self.interval = interval # every 5 minute
        self.prediction_start_time = prediction_start_time # 6:00 a.m., str
        self.prediction_array = prediciotn_array # model output, flattened, [number_of_days, 174]
        self.annomaly_array = annomaly_array # denoised incident label, falttened, [number_of_days, 17?]
        
        df_waze = pd.read_pickle(f"{args.data_dir}/data/{args.county}/processed_data/{args.county}_raw_waze_inc_eval.pkl")

        df_waze_link = df_waze[df_waze.id_tmc == self.link_id]
        df_waze_link['Start time'] = pd.to_datetime(df_waze_link['Start time'], errors='coerce')
        df_waze_link['Closed time'] = pd.to_datetime(df_waze_link['Closed time'], errors='coerce')
        try:
            df_waze_link['Start time'] = df_waze_link['Start time'].apply(lambda x: x.tz_localize(None))
            df_waze_link['Closed time'] = df_waze_link['Closed time'].apply(lambda x: x.tz_localize(None))
        except Exception as e:
            print(f"Error localizing time: {e}")

        self.df_waze_link = df_waze_link

        self.day_n = prediciotn_array.shape[0]
        print('day_number_is', self.day_n)
        busi_date = pd.bdate_range(start=args.start_date, end=args.end_date).date
        self.test_date = busi_date[-self.day_n:]
    
    def _get_time_periods(self, day_array, date_str):
        prediction_start_date_time = dt.strptime(f'{date_str} {self.prediction_start_time}', '%Y-%m-%d %H:%M')
        periods = []
        n = len(day_array)
        i = 0
        while i < n:
            if day_array[i] == 1:
                start_idx = i
                while i < n and day_array[i] == 1:
                    i += 1
                end_idx = i - 1
                start_period = prediction_start_date_time + start_idx * self.interval
                end_period = prediction_start_date_time + (end_idx + 1) * self.interval
                periods.append((start_period, end_period))
            i += 1
        return periods
    
    def _generate_day_incident_periods(self, date_str):
        
        given_date = dt.strptime(date_str, '%Y-%m-%d')
        start_time = given_date.replace(hour=6, minute=0, second=0) # edit as needed
        end_time = given_date.replace(hour=20, minute=25, second=0) # edit as needed
        df_filtered = self.df_waze_link[(self.df_waze_link['Start time'].dt.date == given_date.date()) | (self.df_waze_link['Closed time'].dt.date == given_date.date())]
        incident_periods = []
        
        for _, row in df_filtered.iterrows():
            incident_start = max(row['Start time'], start_time)
            incident_end = min(row['Closed time'], end_time)
            if incident_start < incident_end:
                incident_periods.append((incident_start, incident_end))
        
        if not incident_periods:
            return []
        
        incident_periods.sort()
        merged_periods = []
        current_start, current_end = incident_periods[0]
        for start, end in incident_periods[1:]:
            if start <= current_end:
                current_end = max(current_end, end)
            else:
                merged_periods.append((current_start, current_end))
                current_start, current_end = start, end
        
        merged_periods.append((current_start, current_end))
        merged_periods = [(pd.Timestamp(start).to_pydatetime(), pd.Timestamp(end).to_pydatetime()) for start, end in merged_periods]
        return merged_periods
    
    def _is_overlap(self, period1, period2):
        return period1[0] < period2[1] and period1[1] > period2[0]
    
    
    def _check_far_fp(self,  report_period, alert_period):
        alert_number = len(alert_period)
        false_postive_case = []
        false_alert_number = 0
        for alert in alert_period:
            true_alert = False
            for report in report_period:
                if self._is_overlap(alert, report):
                    true_alert = True
            if not true_alert:
                false_alert_number += 1
                false_postive_case.append(alert)
        return alert_number, false_alert_number, false_postive_case
    

    def _check_sfar_sfp(self, report_period, alert_period, annomaly_period):
        alert_number = len(alert_period)
        sfalse_postive_case = []
        sfalse_alert_number = 0
        for alert in alert_period:
            true_alert = False
            for report in report_period:
                if self._is_overlap(alert, report):
                    print('report', alert, report)
                    true_alert = True
            for annomaly in  annomaly_period:
                if self._is_overlap(alert, annomaly):
                    print('annomaly', alert, annomaly)
                    true_alert = True
            if not true_alert:
                sfalse_alert_number += 1
                sfalse_postive_case.append(alert)
        return alert_number, sfalse_alert_number, sfalse_postive_case
    
    def _check_dr_mttd_fn(self, report_period, alert_period):

        report_number = len(report_period)
        total_ttd = 0
        detected_number = 0
        fail_to_detect_case = []
        
        Detected_Report_Alert_Dict = {} # key, report_id; value, alerts
        Detected_Report_Dict = {}
        
        for report_id in range(len(report_period)):
            report = report_period[report_id]
            detected = False
            ttd_list = []
            for alert in alert_period:
                if self._is_overlap(alert, report):

                    if report_id in Detected_Report_Alert_Dict.keys():
                        Detected_Report_Alert_Dict[report_id].append(alert)
                    else:
                        Detected_Report_Alert_Dict[report_id] = []
                        Detected_Report_Alert_Dict[report_id] = [alert]
                        Detected_Report_Dict[report_id] = []
                        Detected_Report_Dict[report_id] = [report]

                    detected = True
                    time_difference = alert[0] - report[0]
                    time_difference_in_minute = time_difference.total_seconds() / 60
                    ttd_list.append(time_difference_in_minute)
            
            if detected:
                ttd = min(ttd_list)
                print('detected!!!!!!!', ttd_list, ttd)
                total_ttd += ttd
                detected_number += 1

            else:
                fail_to_detect_case.append(report)
        
        if detected_number != len(list(Detected_Report_Dict.keys())):
            raise ValueError('Check _check_dr_mttd_fn Function')

        return report_number, detected_number, total_ttd, fail_to_detect_case, Detected_Report_Dict, Detected_Report_Alert_Dict
    
    def _check_sdr_smttd_sfn(self, report_period, alert_period, annomaly_period):
        signficant_report_period = []
        for report in report_period:
            for annomaly in annomaly_period:
                if self._is_overlap(report, annomaly):
                    signficant_report_period.append(report)
                    break 

        s_report_number = len(signficant_report_period)
        s_total_ttd = 0
        s_detected_number = 0
        s_fail_to_detect_case = []

        Detected_Report_Alert_Dict = {} # key, report_id; value, alerts
        Detected_Report_Dict = {}

        for report_id in range(len(signficant_report_period)):

            report = signficant_report_period[report_id]

            detected = False
            ttd_list = []
            for alert in alert_period:
                if self._is_overlap(alert, report):

                    if report_id in Detected_Report_Alert_Dict.keys():
                        Detected_Report_Alert_Dict[report_id].append(alert)
                    else:
                        Detected_Report_Alert_Dict[report_id] = []
                        Detected_Report_Alert_Dict[report_id] = [alert]
                        Detected_Report_Dict[report_id] = []
                        Detected_Report_Dict[report_id] = [report]

                    detected = True
                    time_difference = alert[0] - report[0]
                    time_difference_in_minute = time_difference.total_seconds() / 60
                    ttd_list.append(time_difference_in_minute)
            if detected:
                ttd = min(ttd_list)
                s_total_ttd += ttd
                s_detected_number += 1
            else:
                s_fail_to_detect_case.append(report)
            
        if s_detected_number != len(list(Detected_Report_Dict.keys())):
            raise ValueError('Check _check_sdr_smttd_sfn Function')
        
        return s_report_number, s_detected_number, s_total_ttd, s_fail_to_detect_case, Detected_Report_Dict, Detected_Report_Alert_Dict
    
    def check_day_result(self, day_pred_array, date_str):
        day_report_merged_period = self._generate_day_incident_periods(date_str)
        day_alart_period = self._get_time_periods(day_pred_array, date_str)
        day_report_number, day_detected_number, day_total_ttd, day_fail_to_detect_cases, Day_Detected_Report_Dict, Day_Detected_Report_Alert_Dict = self._check_dr_mttd_fn(day_report_merged_period, day_alart_period)
        day_alert_number, day_false_alert_number, day_false_postive_cases = self._check_far_fp(day_report_merged_period, day_alart_period)
        return day_report_number, day_detected_number, day_total_ttd, day_alert_number, day_false_alert_number, day_fail_to_detect_cases, day_false_postive_cases, Day_Detected_Report_Dict, Day_Detected_Report_Alert_Dict
    
    def check_day_result_s(self, day_pred_array, day_annomlay_array, date_str):
        day_report_merged_period = self._generate_day_incident_periods(date_str)
        day_alart_period = self._get_time_periods(day_pred_array, date_str)
        day_annom_period = self._get_time_periods(day_annomlay_array, date_str)
        day_report_number, day_detected_number, day_total_ttd, day_fail_to_detect_cases, Day_Detected_Report_Dict, Day_Detected_Report_Alert_Dict = self._check_sdr_smttd_sfn(day_report_merged_period, day_alart_period, day_annom_period)
        day_alert_number, day_false_alert_number, day_false_postive_cases = self._check_sfar_sfp(day_report_merged_period, day_alart_period, day_annom_period)
        return day_report_number, day_detected_number, day_total_ttd, day_alert_number, day_false_alert_number, day_fail_to_detect_cases, day_false_postive_cases, Day_Detected_Report_Dict, Day_Detected_Report_Alert_Dict
    
    def check_overall_results(self):
        total_report_number, total_detected_number, total_ttd, total_alert_number, total_false_alert_number = 0, 0, 0, 0, 0
        all_fail_to_detect_cases = []
        all_false_postive_cases = []

        Detected_Report_Dict = {}
        Detected_Report_Alert_Dict = {}

        for i in range(len(self.test_date)):
            day_pred_array = self.prediction_array[i]
            date_str = self.test_date[i].strftime("%Y-%m-%d")
            day_report_number, day_detected_number, day_total_ttd, day_alert_number, day_false_alert_number, day_fail_to_detect_cases, day_false_postive_cases, Day_Detected_Report_Dict, Day_Detected_Report_Alert_Dict = self.check_day_result(day_pred_array, date_str)
            total_report_number += day_report_number
            total_detected_number += day_detected_number
            total_ttd += day_total_ttd 
            total_alert_number += day_alert_number 
            total_false_alert_number += day_false_alert_number
            all_fail_to_detect_cases.extend(day_fail_to_detect_cases)
            all_false_postive_cases.extend(day_false_postive_cases)

            report_id_list = list(Day_Detected_Report_Dict.keys())

            for j in range(day_detected_number):
                Detected_Report_Dict[total_detected_number-day_detected_number+j] = []
                Detected_Report_Alert_Dict[total_detected_number-day_detected_number+j] = []
                report_id = report_id_list[j]
                Detected_Report_Dict[total_detected_number-day_detected_number+j] = Day_Detected_Report_Dict[report_id]
                Detected_Report_Alert_Dict[total_detected_number-day_detected_number+j] = Day_Detected_Report_Alert_Dict[report_id]
        
        print('total_detected_number', total_detected_number)
        print('total_report_number', total_report_number)
        if total_report_number != 0:
            DR = total_detected_number/total_report_number
        else:
            DR = 0
        
        if total_detected_number !=0:
            MTTD = total_ttd/total_detected_number
        else:
            MTTD = 0

        if total_alert_number!= 0:
            FAR = total_false_alert_number/total_alert_number
        else:
            FAR = 0
        
        return DR, FAR, MTTD, all_fail_to_detect_cases, all_false_postive_cases, Detected_Report_Dict, Detected_Report_Alert_Dict
    
    def check_overall_s_results(self):
        total_report_number, total_detected_number, total_ttd, total_alert_number, total_false_alert_number = 0, 0, 0, 0, 0
        all_fail_to_detect_cases = []
        all_false_postive_cases = []

        Detected_Report_Dict = {}
        Detected_Report_Alert_Dict = {}

        for i in range(len(self.test_date)):
            day_pred_array = self.prediction_array[i]
            day_anno_array = self.annomaly_array[i]
            date_str = self.test_date[i].strftime("%Y-%m-%d")
            day_report_number, day_detected_number, day_total_ttd, day_alert_number, day_false_alert_number, day_fail_to_detect_cases, day_false_postive_cases, Day_Detected_Report_Dict, Day_Detected_Report_Alert_Dict = self.check_day_result_s(day_pred_array, day_anno_array, date_str)
            total_report_number += day_report_number
            total_detected_number += day_detected_number
            total_ttd += day_total_ttd 
            total_alert_number += day_alert_number 
            total_false_alert_number += day_false_alert_number
            all_fail_to_detect_cases.extend(day_fail_to_detect_cases)
            all_false_postive_cases.extend(day_false_postive_cases)


            report_id_list = list(Day_Detected_Report_Dict.keys())

            for j in range(day_detected_number):
                Detected_Report_Dict[total_detected_number-day_detected_number+j] = []
                Detected_Report_Alert_Dict[total_detected_number-day_detected_number+j] = []
                report_id = report_id_list[j]
                Detected_Report_Dict[total_detected_number-day_detected_number+j] = Day_Detected_Report_Dict[report_id]
                Detected_Report_Alert_Dict[total_detected_number-day_detected_number+j] = Day_Detected_Report_Alert_Dict[report_id]
        
        print('total_detected_number', total_detected_number)
        print('total_report_number', total_report_number)
        if total_report_number != 0:
            DR = total_detected_number/total_report_number
        else:
            DR = 0
        
        if total_detected_number !=0:
            MTTD = total_ttd/total_detected_number
        else:
            MTTD = 0

        if total_alert_number!= 0:
            FAR = total_false_alert_number/total_alert_number
        else:
            print('no alert the whole day!!!!!!!!!')
            FAR = 0
        
        return DR, FAR, MTTD, all_fail_to_detect_cases, all_false_postive_cases, Detected_Report_Dict, Detected_Report_Alert_Dict

In [9]:
args.link_id = '110+04483'
args.county = "TSMO"
args.model_path = 'E:/two_stage_model'
args.start_date = dt(2022, 2, 14) #dt(2022, 2, 1)
args.end_date =  dt(2023, 2, 12) #dt(2024, 1, 31)
args.number_of_business_day = 260
use_GAN = False
test_alert = load_SVM_pred(use_GAN, args)
test_annomaly = load_test_annomaly(args)
IAD = IncidentDetectionAnalysis(args, test_alert, test_annomaly)
DR, FAR, MTTD, all_fail_to_detect_cases, all_false_postive_cases, Detected_Report_Dict, Detected_Report_Alert_Dict = IAD.check_overall_results()
print('DR', DR, 'FAR', FAR, 'MTTD', MTTD)
SDR, SFAR, SMTTD, Sall_fail_to_detect_cases, Sall_false_postive_cases, SDetected_Report_Dict, SDetected_Report_Alert_Dict = IAD.check_overall_s_results()
print('SDR', SDR, 'SFAR', SFAR, 'SMTTD', SMTTD)

day_number_is 26
detected!!!!!!! [-8.166666666666666] -8.166666666666666
detected!!!!!!! [-66.38333333333334] -66.38333333333334
detected!!!!!!! [-17.566666666666666] -17.566666666666666
detected!!!!!!! [-8.183333333333334] -8.183333333333334
detected!!!!!!! [-65.13333333333334] -65.13333333333334
DR 0.7142857142857143 FAR 0.975609756097561 MTTD -33.086666666666666
SDR 1.0 SFAR 0.9451219512195121 SMTTD -24.762500000000003


C:\Users\haowu\AppData\Local\Temp\ipykernel_512216\3925642866.py:19: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_waze_link['Start time'] = pd.to_datetime(df_waze_link['Start time'], errors='coerce')
c:\Users\haowu\Anaconda3\envs\traffic\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\haowu\AppData\Local\Temp\ipykernel_512216\3925642866.py:20: FutureWarning: In a future version of pan

In [31]:
args.link_id = '110+04483'
args.county = "TSMO"
args.model_path = 'E:/two_stage_model'
args.start_date = dt(2022, 2, 14) #dt(2022, 2, 1)
args.end_date =  dt(2023, 2, 12) #dt(2024, 1, 31)
args.number_of_business_day = 260
use_GAN = True
test_alert = load_SVM_pred(use_GAN, args)
test_annomaly = load_test_annomaly(args)
IAD = IncidentDetectionAnalysis(args, test_alert, test_annomaly)
DR, FAR, MTTD, all_fail_to_detect_cases, all_false_postive_cases, Detected_Report_Dict, Detected_Report_Alert_Dict = IAD.check_overall_results()
print('DR', DR, 'FAR', FAR, 'MTTD', MTTD)
SDR, SFAR, SMTTD, Sall_fail_to_detect_cases, Sall_false_postive_cases, SDetected_Report_Dict, SDetected_Report_Alert_Dict = IAD.check_overall_s_results()
print('SDR', SDR, 'SFAR', SFAR, 'SMTTD', SMTTD)

day_number_is 26
detected!!!!!!! [-7.566666666666666] -7.566666666666666
detected!!!!!!! [11.816666666666666] 11.816666666666666
detected!!!!!!! [-45.13333333333333] -45.13333333333333
DR 0.42857142857142855 FAR 0.6 MTTD -13.627777777777775
SDR 0.75 SFAR 0.2 SMTTD -13.627777777777775


C:\Users\haowu\AppData\Local\Temp\ipykernel_293732\3925642866.py:19: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_waze_link['Start time'] = pd.to_datetime(df_waze_link['Start time'], errors='coerce')
c:\Users\haowu\Anaconda3\envs\traffic\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\haowu\AppData\Local\Temp\ipykernel_293732\3925642866.py:20: FutureWarning: In a future version of pan